In [1]:
import sys
import os
os.chdir("../")

In [2]:
import numpy as np
import pandas as pd
import data
from tqdm.auto import tqdm
tqdm.pandas()
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_colwidth = -1

#### sort order --> filter selection

price only --> Sort by Price

distance only --> Sort By Distance

rating only --> Sort By Rating

price and recommended --> Best Value

rating and recommended --> Focus on Rating

distance and recommended --> Focus on Distance

our recommendations --> Sort by Popularity

In [3]:
train_df = data.train_df('small', cluster='cluster_recurrent')
print(train_df.shape)

(179255, 13)


In [4]:
train_df[train_df.current_filters.notnull()]

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence
176,03P4VFKK12UO,325fafb5fa450,1541107500,54,clickout item,65685,US,"Bakersfield, USA",desktop,Very Good Rating|5 Star|4 Star|Hotel|Motel|Resort|Hostal (ES)|3 Star,1306936|56482|2842358|6881276|65685|63259|65399|1320460|360566|376756|885381|60769|2141484|375876|829246|58909|360571|1298518|63542|71926|376076|57803|376181|1117192|359326,178|104|110|94|57|96|46|61|48|35|50|38|59|44|57|56|55|45|146|41|44|44|39|38|126,1.0
177,03P4VFKK12UO,325fafb5fa450,1541107538,55,clickout item,1320460,US,"Bakersfield, USA",desktop,Very Good Rating|5 Star|4 Star|Hotel|Motel|Resort|Hostal (ES)|3 Star,1306936|56482|2842358|6881276|65685|63259|65399|1320460|360566|376756|885381|60769|2141484|375876|829246|58909|360571|1298518|63542|71926|376076|57803|376181|1117192|359326,178|104|110|94|57|96|46|61|48|35|50|38|59|44|57|56|55|45|146|41|44|44|39|38|126,1.0
445,0NA8E4AD2VY7,336d9a46b826f,1541078452,91,clickout item,1091180,RU,"Yekaterinburg, Russia",desktop,Free WiFi (Combined)|Breakfast Included,1045290|3135752|1372961|1369024|3384308|1371070|1191128|320366|2589366|1373129|1091180|3404929|5813690|939106|940286|3183138|1370286|384511|5127144|2740100|939964|1327476|3961622|939114|1370246,44|87|30|62|73|59|85|43|68|31|35|48|59|56|42|116|29|79|38|42|28|47|28|50|67,1.0
446,0NA8E4AD2VY7,336d9a46b826f,1541078493,92,clickout item,1091180,RU,"Yekaterinburg, Russia",desktop,Free WiFi (Combined)|Breakfast Included,1045290|3135752|1372961|1369024|3384308|1371070|1191128|320366|2589366|1373129|1091180|3404929|5813690|939106|940286|3183138|1370286|384511|5127144|2740100|939964|1327476|3961622|939114|1370246,44|87|30|62|73|59|85|43|68|31|35|48|59|56|42|116|29|79|38|42|28|47|28|50|67,1.0
466,0NA8E4AD2VY7,336d9a46b826f,1541079107,112,clickout item,3404929,RU,"Yekaterinburg, Russia",desktop,Free WiFi (Combined)|Breakfast Included,1045290|3135752|1372961|1369024|3384308|1371070|1191128|320366|2589366|1373129|1091180|3404929|5813690|939106|940286|3183138|1370286|384511|5127144|2740100|939964|1327476|3961622|939114|1370246,44|87|30|62|73|59|85|43|68|31|35|48|59|56|42|116|29|79|38|42|28|47|28|50|67,1.0
487,0O1CTEP95YKQ,af793494b5f84,1541113474,19,clickout item,63051,CO,"Las Vegas, USA",mobile,Breakfast Included|Sort by Price,69086|8533|925367|58949|61049|8581|8517|77520|8531|4169220|63051|397316|840726|2851635|1493341|8804492|62454|450176|102082|148954|8602|441851|63858|8516|8510248,59|66|78|86|87|90|90|92|93|98|99|100|106|108|111|111|112|120|122|123|125|136|138|140|142,1.0
488,0O1CTEP95YKQ,af793494b5f84,1541113509,20,clickout item,2851635,CO,"Las Vegas, USA",mobile,Breakfast Included|Sort by Price,69086|8533|925367|58949|61049|8581|8517|77520|8531|4169220|63051|397316|840726|2851635|1493341|8804492|62454|450176|102082|148954|8602|441851|63858|8516|8510248,59|66|78|86|87|90|90|92|93|98|99|100|106|108|111|111|112|120|122|123|125|136|138|140|142,1.0
490,0O1CTEP95YKQ,af793494b5f84,1541113537,22,clickout item,8602,CO,"Las Vegas, USA",mobile,Breakfast Included|Sort by Price,69086|8533|925367|58949|61049|8581|8517|77520|8531|4169220|63051|397316|840726|2851635|1493341|8804492|62454|450176|102082|148954|8602|441851|63858|8516|8510248,59|66|78|86|87|90|90|92|93|98|99|100|106|108|111|111|112|120|122|123|125|136|138|140|142,1.0
539,0Q3605EUBZG3,641a409c90fac,1541095202,44,clickout item,4041114,BR,"Rio de Janeiro, Brazil",mobile,Best Value,2303306|4041114|5830856|111534|109353|5723646|1208722|1255025|1670475|1055330|655691|2284048|104681|104809|130772|5822734|1668389|1771225|2342182|104800|1329092|9719914|2627243|3249257|109332,72|29|122|98|187|86|66|124|112|104|118|107|146|179|136|113|102|115|96|147|136|36|65|93|115,1.0
542,0Q3605EUBZG3,641a409c90fac,1541095238,47,clickout item,9112068,BR,"Rio de Janeiro, Brazil",mobile,Best Value,5016850|2068758|1771981|2303306|2733700|3189448|5849680|1179630|1319947|1770143|925941|926343|8428074|109331|829671|3906960|4094062|5

In [4]:
#change_sort_filters = set(['sort by price', 'sort by distance', 'sort by rating', 'sort by popularity',
#                    'focus on rating', 'focus on distance', 'best value'])

change_sort_filters = set(['sort by price','best value', 
                           'sort by rating','focus on rating',
                           'sort by popularity'])

classes = ['sort_rating', 'sort_pop', 'sort_price']
mapping = {
    'sort by price':      [0,0,1],
    'best value':         [0,1,1],
    'sort by rating':     [1,0,0],
    'focus on rating':    [1,1,0],
    'sort by popularity': [0,1,0],
}

### Take the interactions with some change_of_sort_order filters

In [5]:
rows = train_df[(train_df.action_type == 'clickout item') & train_df.current_filters.notnull()]
rows = rows[['current_filters']]
# filter the filters by the sort filters and re-cast to list
rows['filters_list'] = rows['current_filters'].str.lower().str.split('|')\
                                .progress_apply(lambda x: list(set(x) & change_sort_filters))
rows = rows.drop(['current_filters'], axis=1,)
#rows['filters_list'] = rows['filters_list'].fillna('')
#rows['filters_list'] = rows[rows['filters_list'].str.len() > 0]
rows = rows[rows['filters_list'].str.len() > 0] #.apply(lambda x: x['filters_list'].values[0])
rows['filters_list'] = rows['filters_list'].apply(lambda x: x[0])

In [6]:
rows.head(10)

,filters_list
487,sort by price
488,sort by price
490,sort by price
539,best value
542,best value
642,sort by price
648,sort by price
651,sort by price
1882,best value
1883,best value


### Encode the change-of-sort-order filters

In [7]:
# iterate over the interactions
matrix = np.zeros((rows.shape[0], 3), dtype='int8')
k = 0
for fl in tqdm(rows['filters_list'].values):
    matrix[k,:] = mapping[fl]
    k += 1

In [8]:
matrix

array([[0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       ...,
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1]], dtype=int8)

In [9]:
# add the 3 new columns
for i,col_name in enumerate(classes):
    rows[col_name] = matrix[:,i]

In [10]:
rows

,filters_list,sort_rating,sort_pop,sort_price
487,sort by price,0,0,1
488,sort by price,0,0,1
490,sort by price,0,0,1
539,best value,0,1,1
542,best value,0,1,1
642,sort by price,0,0,1
648,sort by price,0,0,1
651,sort by price,0,0,1
1882,best value,0,1,1
1883,best value,0,1,1


In [5]:
from extract_features.rnn.change_sort_order_filters import ChangeSortOrderFilters
feature = ChangeSortOrderFilters()

In [6]:
aa = feature.join_to(train_df)
aa


feature 'change_sort_order_filters' does not exist. want to create? (y/n)
y
caching df_full...


/Users/federico/miniconda3/envs/recsys/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Done!



Total interactions: 240028



dataset/preprocessed/no_cluster/full/feature/change_sort_order_filters folder created
change_sort_order_filters feature read


,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices,frequence,sort_rating,sort_pop,sort_price
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0,0,0,0
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,3.0,0,0,0
4,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0,0,0,0
5,00RL8Z82B2Z1,aff3928535f48,1541037532,6,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0,0,0,0
6,00RL8Z82B2Z1,aff3928535f48,1541037532,7,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0,0,0,0
7,00RL8Z82B2Z1,aff3928535f48,1541037532,8,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0,0,0,0
8,00RL8Z82B2Z1,aff3928535f48,1541037542,9,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,5.0,0,0,0
13,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3400638|1253714|3367857|5100540|1088584|666916|54833|2922310|9711560|109038|666856|10077318|1431482|129343|6339822|6806806|1041528|109013|3909420|55088|3095758|109018|54885|1257342|2595006,95|66|501|112|95|100|101|72|82|56|56|143|70|25|71|162|73|143|188|118|77|131|143|49|165,1.0,0,0,0
14,00RL8Z82B2Z1,aff3928535f48,1541038469,15,search for poi,Surry Hills,AU,"Sydney, Australia",mobile,NaN,NaN,NaN,1.0,0,0,0
15,00RL8Z82B2Z1,aff3928535f48,1541038485,16,clickout item,1257342,AU,"Sydney, Australia",mobile,NaN,55109|129343|54824|2297972|109014|1257342|1031578|109018|1332971|666916|54833|54885|2237222|10077318|1166793|9132132|1474297|3909420|6622154|55091|8444418|54804|2050977|666936|55088,162|25|150|143|101|49|118|131|18|100|101|143|51|143|123|36|66|188|124|138|94|137|180|112|118,1.0,0,0,0
